In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

questions_types = {
    'question_id': 'int16',
    'bundle_id': 'int16',
    'correct_answer': 'int8',
    'part': 'int8',
    'tags': 'string'
}
questions = pd.read_csv('/kaggle/input/riiid-test-answer-prediction/questions.csv', dtype = questions_types) # wszystkie dane ładowane do pamięci
questions.info()

In [ ]:
train_types = {
    'row_id': 'int64',
    'timestamp': 'int64',
    'user_id': 'int32',
    'content_id': 'int16',
    'content_type_id': 'int8',
    'task_container_id': 'int16',
    'user_answer': 'int8',
    'answered_correctly': 'int8',
    'prior_question_elapsed_time': 'float32', 
    'prior_question_had_explanation': 'boolean'
}        


train = pd.read_csv('/kaggle/input/riiid-test-answer-prediction/train.csv', dtype = train_types) # wszystkie dane ładowane do pamięci
# train = pd.read_csv('/kaggle/input/riiid-test-answer-prediction/train.csv', dtype = train_types, nrows=1000) # pierwsze 1000 wierszy

In [ ]:
print('W zbiorze istnieje ' + str(questions.shape[0]) + ' pytań')

In [ ]:
print('Przykładowe tagi:')
print(questions.loc[0].tags)
print(questions.loc[100].tags)
print(questions.loc[220].tags)

In [ ]:
questions_no_tags = questions[questions.tags.isna()]
print('W zbiorze istnieje ' + str(questions_no_tags.shape[0]) + ' pytanie bez tagów:')
print(questions_no_tags);
# Ma to duże, znaczenie, zależy w przy przetwarzaniu danych sprawdzić czy tagi nie są NA


In [ ]:
questions_with_tags = questions[questions.tags.isna() == False]

tags = [x.split() for x in questions_with_tags.tags.values]
tags_set = set([item for elem in tags for item in elem])
print('W zbiorze istnieje ' + str(len(tags_set)) + ' różnych tagów')



In [ ]:
# Zmiana typu tagów z str na listę:
questions.tags = [tags.split() if type(tags) is str else tags for tags in questions.tags.values]
questions.head(5)

In [ ]:
no_lectures = train[train.answered_correctly != -1] # wykluczenie wykładów
x = no_lectures.groupby(['content_id']).size()
correct = no_lectures.groupby(['content_id']).agg({'answered_correctly': ['mean']})
correct = pd.DataFrame(correct)


df = no_lectures.groupby(['content_id']).agg({'answered_correctly': ['mean', 'count', 'sum']})
df = df['answered_correctly']
questions = questions.merge(df, left_on = 'question_id', right_on = 'content_id', how = "left")
questions = questions.fillna({'mean': '0.0', 'count': '0', 'sum': '0'})
#questions['sum'] = questions['sum'].astype('int8')
#questions['count'] = questions['count'].astype('int8')
questions = questions.rename(columns={'mean': 'Srednia poprawnosc', 'count': 'Ile razy zadane', 'sum': 'Ilosc poprwnych'})
questions.head()


In [ ]:
tags_list = list(tags_set)
tags_df = pd.DataFrame()
questions_with_tags = questions[questions.tags.isna() == False]
for tag in tags_list:
    df = questions_with_tags[questions_with_tags.tags.apply(lambda l: tag in l)]
    tmp_df = df.agg({'Ile razy zadane': ['sum'], 'Ilosc poprwnych': ['sum']})
    tmp_df['tag'] = tag
    tmp_df['Ilosc pytań z tym tagiem'] = len(df)
    tmp_df = tmp_df.set_index('tag')
    tags_df = tags_df.append(tmp_df)
    
tags_df['Procent poprawności'] = tags_df['Ilosc poprwnych'] / tags_df['Ile razy zadane']
tags_df = tags_df.sort_values(by='Procent poprawności')
tags_df.head()

In [ ]:
tags_df = tags_df.sort_values(by='Procent poprawności')
how_much = 5
head = tags_df.head(how_much)
tail = tags_df.tail(how_much)

allin = head.append(tail)
x = list(allin['Procent poprawności'].index)
y = list(allin['Procent poprawności'].values)

fig = plt.figure(figsize = (13, 5)) 
plt.bar(x, y, color=['red'] * how_much + ['lightgreen'] * how_much)
  
plt.xlabel('Tag') 
plt.ylabel('Procent poprawności') 
plt.show() 

In [ ]:
tags_df = tags_df.sort_values(by='Ile razy zadane')
how_much = 15
head = tags_df.head(how_much)
tail = tags_df.tail(how_much)

allin = head.append(tail)
x = list(allin['Ile razy zadane'].index)
y = list(allin['Ile razy zadane'].values)

fig = plt.figure(figsize = (13, 5)) 
plt.bar(x, y, color=['red'] * how_much + ['lightgreen'] * how_much)
  
plt.xlabel('Tag') 
plt.ylabel('Ile razy zadane było pytanie z tym tagiem') 
plt.show() 

In [ ]:
# Parts = części testu TOEIC, parts 1-4 słuchanie, parts 5-7 czytanie
# Na wykresie: różowe - słuchanie, niebieski - czytanie
fig = plt.figure(figsize=(13,5))
questions.groupby("part").count()['question_id'].plot.bar(color=['pink'] * 4 + ['lightblue'] * 3)
plt.title("Ile pytań zostało zadanych w danej części testu TOEIC")
plt.xlabel("Część testu")
plt.xticks(rotation=0)
plt.show()

In [ ]:
# Na wykresie: różowe - słuchanie, niebieski - czytanie
parts = questions.groupby('part').agg({'Ile razy zadane': ['sum'], 'Ilosc poprwnych': ['sum']})
parts['Procent poprawności'] = parts['Ilosc poprwnych'] / parts['Ile razy zadane']
parts['Procent poprawności']

x = list(parts['Procent poprawności'].index)
y = list(parts['Procent poprawności'].values)

fig = plt.figure(figsize = (13, 5)) 
plt.bar(x, y, color=['pink'] * 4 + ['lightblue'] * 3)
  
plt.xlabel('Część testu') 
plt.ylabel('Procent poprawnych odpowiedzi') 
plt.show() 

In [ ]:
bundles = questions['bundle_id']
bundles.nunique()


In [ ]:
bundles = questions.groupby('bundle_id', as_index=True).agg({'Ile razy zadane': ['sum'], 'Ilosc poprwnych': ['sum']})
bundles['Procent poprawności'] = bundles['Ilosc poprwnych'] / bundles['Ile razy zadane']
bundles = bundles.sort_values(by='Procent poprawności')

how_much = 4

head = bundles.head(how_much)
tail = bundles.tail(how_much)

allin = head.append(tail)
x = list(allin['Procent poprawności'].index)
y = list(allin['Procent poprawności'].values)
fig = plt.figure(figsize = (13, 5)) 
plt.bar(x, y, width=60)
  
plt.xlabel('Bundle id') 
plt.ylabel('Procent poprawnych odpowiedzi') 
plt.show() 


In [ ]:
bundles = questions.groupby('bundle_id').agg({'question_id': ['count']})
bundles_by_count = bundles['question_id'].sort_values(by='count')
print(bundles_by_count)
# maksymalnie 5 pytań występuje w danym bundle



In [ ]:
index = bundles_by_count.index[len(bundles_by_count) - 1]
questions[questions.bundle_id == index]

In [ ]:
index = bundles_by_count.index[len(bundles_by_count) - 2]
questions[questions.bundle_id == index]

In [ ]:
index = bundles_by_count.index[len(bundles_by_count) - 3]
questions[questions.bundle_id == index]

In [ ]:
index = bundles_by_count.index[len(bundles_by_count) - 4]
questions[questions.bundle_id == index]

In [ ]:
index = bundles_by_count.index[len(bundles_by_count) - 5]
questions[questions.bundle_id == index]